# CIFAR10 Results

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

pd.options.display.float_format = "{:,.2f}".format

In [2]:
def extract_metrics_gpu(filename): 
    f = open(filename, 'r')
    lines = f.readlines() 
    ram_init = None
    ram_peak = None
    cpu_init = None 
    cpu_end = None 
    for line in lines: 
        if "=> Average precision" in line: 
            accuracy = float(line.split(" ")[-1])
        if "=> Total training time" in line: 
            training_time = float(line.split(" ")[-2])
        if "RAM BEFORE TRAINING:" in line: 
            ram_init = float(line.split(" ")[-1])
        if "PEAK TRAINING RAM:" in line: 
            ram_peak = float(line.split(" ")[-1])
        if "CPU BEFORE TRAINING:" in line: 
            cpu_init = eval(line.split(":")[-1])
        if "CPU BEFORE EVALUATION:" in line: 
            cpu_end = eval(line.split(":")[-1])
        if "GPU BEFORE EVALUATION:" in line: 
            gpu_u, gpu_mem = eval(line.split(":")[-1])
        
    cpu_u, cpu_k = (cpu_end[0] - cpu_init[0], cpu_end[1] - cpu_init[1])
    return (accuracy * 100, training_time, (ram_peak-ram_init) * 1024, cpu_u + cpu_k, gpu_u, gpu_mem)

In [3]:
def calculate_metrics_gpu(files): 
    mlp_metrics = dict()
    cnn_metrics = dict()
    
    for file in files: 
        arch, method, i = file.split("_") 
        metrics = mlp_metrics if arch == "mlp" else cnn_metrics
        
        if method in metrics: 
            metrics[method].append(extract_metrics_gpu(file))
        else: 
            metrics[method] = [extract_metrics_gpu(file)] 
    
    return (mlp_metrics, cnn_metrics)

In [4]:
def get_means(metrics): 
    return {method: [sum(x)/len(x) for x in list(zip(*metrics[method]))] for method in metrics}

# 1. Task-IL

In [5]:
%cd /home/nc528/Desktop/Projects/MPhilSupervisions/ss2719/research_project/experiments/cifar10/2022-01-17-17-28/task_gpu_fi

logs = !ls
# logs = !ls | grep "cnn\|mlp"

mlp_metrics, cnn_metrics = calculate_metrics_gpu(logs)
mlp_metrics = get_means(mlp_metrics)
cnn_metrics = get_means(cnn_metrics)

/auto/homes/nc528/Desktop/Projects/MPhilSupervisions/ss2719/research_project/experiments/cifar10/2022-01-17-17-28/task_gpu_fi


## 1.1. MLP Results

In [6]:
df = pd.DataFrame(mlp_metrics, index=['Accuracy (%)',
                                 'Training time (s)', 
                                 'RAM Usage (MB)', 
                                 'CPU Usage (s)', 
                                 'GPU Usage (%)', 
                                 'GPU Memory (MB)']).filter(["nr", "lr", "gr", "lgr", "grd", "lgrd"])



## 1.2. CNN Results

In [7]:
pd.DataFrame(cnn_metrics, index=['Accuracy (%)', 
                                 'Training time (s)', 
                                 'RAM Usage (MB)', 
                                 'CPU Usage (s)', 
                                 'GPU Usage (%)', 
                                 'GPU Memory (MB)']).filter(["nr", "lr", "gr", "lgr", "grd", "lgrd"])

,nr,lr,gr,lgr,grd,lgrd
Accuracy (%),67.83,66.53,68.23,66.80,69.42,69.11
Training time (s),76.43,73.37,107.23,101.90,108.57,103.87
RAM Usage (MB),1.99,2.61,2.13,2.18,2.13,2.26
CPU Usage (s),135.36,130.82,163.98,157.82,163.20,158.42
GPU Usage (%),7.62,6.21,18.97,12.12,18.92,11.52
GPU Memory (MB),90.00,12.00,160.00,4.00,160.00,4.00


# 2. Class-IL 

In [8]:
%cd /home/nc528/Desktop/Projects/MPhilSupervisions/ss2719/research_project/experiments/cifar10/2022-01-14-16-32/class_gpu_fi

logs = !ls | grep "cnn\|mlp"

mlp_metrics, cnn_metrics = calculate_metrics_gpu(logs)
mlp_metrics = get_means(mlp_metrics)
cnn_metrics = get_means(cnn_metrics)

/auto/homes/nc528/Desktop/Projects/MPhilSupervisions/ss2719/research_project/experiments/cifar10/2022-01-14-16-32/class_gpu_fi


## 2.1. MLP Results

In [9]:
pd.DataFrame(mlp_metrics, index=['Accuracy (%)', 
                                 'Training time (s)', 
                                 'RAM Usage (MB)', 
                                 'CPU Usage (s)', 
                                 'GPU Usage (%)', 
                                 'GPU Memory (MB)']).filter(["nr", "lr", "gr", "lgr", "grd", "lgrd"])

,nr,lr,gr,lgr,grd,lgrd
Accuracy (%),26.18,26.51,12.53,10.39,16.61,14.38
Training time (s),85.13,82.47,113.80,108.57,116.37,107.97
RAM Usage (MB),0.52,1.85,2.89,3.90,3.50,4.14
CPU Usage (s),148.40,145.72,176.49,171.88,181.21,170.17
GPU Usage (%),5.81,4.15,16.31,9.24,16.28,8.91
GPU Memory (MB),164.00,18.00,184.00,6.00,184.00,6.00


## 2.2. CNN Results

In [10]:
pd.DataFrame(cnn_metrics, index=['Accuracy (%)', 
                                 'Training time (s)', 
                                 'RAM Usage (MB)', 
                                 'CPU Usage (s)', 
                                 'GPU Usage (%)', 
                                 'GPU Memory (MB)']).filter(["nr", "lr", "gr", "lgr", "grd", "lgrd"])

,nr,lr,gr,lgr,grd,lgrd
Accuracy (%),27.70,26.73,12.75,14.39,14.12,13.36
Training time (s),181.83,172.37,239.80,222.83,241.50,231.10
RAM Usage (MB),1.95,3.44,2.05,1.79,1.83,1.76
CPU Usage (s),309.86,303.19,364.62,347.70,366.92,358.53
GPU Usage (%),6.53,5.09,16.90,9.91,16.99,9.57
GPU Memory (MB),138.67,18.67,160.00,4.00,160.00,4.00
